# Question: Writing Viterbi Algorithm for the Primer
Write the Viterbi algorithm to implment Nature Primer.

Here are some suggestions:

a. You can begin by first defining all the parameters, such as states, transition matrix, and emmision matrix etc.

b. You can write a function to exactly calculate the values mentioned in the primer, for example, you can define a function get_log_prob_of_a_given_path ("EEEEEEEEEEEEEEEEEE5IIIIIII", "CTTCATGTGAAAGCAGACGTAAGTCA"). This should output -41.22.

By doing the above two, you earn 1 mark.

Now, you have to implement this in real to get max likely path that would emmit the observed sequence. You MUST note that maximum likely path will just be Es, but that is okay. Implementation is the key.

In [2]:
# Imports
import numpy as np
import math

In [24]:
# Define states, emissions, and transitions
states = ['E', 'I']
observations = ['A', 'C', 'G', 'T']
# Transition log probabilities
transitions = {
    'E': {'E': math.log(0.9), 'I': math.log(0.1)},
    'I': {'E': math.log(0.1), 'I': math.log(0.9)}
}

# Emission log probabilities
emissions = {
    'E': {'A': math.log(0.25), 'C': math.log(0.25), 'G': math.log(0.25), 'T': math.log(0.25)},
    'I': {'A': math.log(0.4),  'C': math.log(0.1),  'G': math.log(0.1),  'T': math.log(0.4)}
}

# Log-probability of a known path and sequence
def log_prob_of_path(state_path, observations):
    assert len(state_path) == len(observations)
    log_prob = emissions[state_path[0]][observations[0]]
    
    for i in range(1, len(state_path)):
        prev_state, curr_state = state_path[i-1], state_path[i]
        transition_score = transitions[prev_state][curr_state]
        emission_score = emissions[curr_state][observations[i]]
        log_prob += transition_score + emission_score
    
    return log_prob

    
#  Viterbi algorithm to find most likely state path
def viterbi_decode(observations):
    sequence_length = len(observations)
    dp_table = [{} for _ in range(sequence_length)]
    back_ptr = [{} for _ in range(sequence_length)]

    # Initialization
    for state in states:
        dp_table[0][state] = emissions[state][observations[0]]
        back_ptr[0][state] = None

    # Dynamic programming loop
    for t in range(1, sequence_length):
        for curr_state in states:
            max_score, best_prev = max(
                (
                    dp_table[t-1][prev_state] + transitions[prev_state][curr_state] + emissions[curr_state][observations[t]],
                    prev_state
                )
                for prev_state in states
            )
            dp_table[t][curr_state] = max_score
            back_ptr[t][curr_state] = best_prev

    # Reconstruct the best path
    last_state = max(dp_table[-1], key=dp_table[-1].get)
    best_path = [last_state]

    for t in range(sequence_length - 1, 0, -1):
        last_state = back_ptr[t][last_state]
        best_path.append(last_state)

    return ''.join(reversed(best_path))


In [25]:
get_log_prob_of_a_given_path("EEEEEEEEEEEEEEEEEEIIIIIIII","CTTCATGTGAAAGCAGACGTAAGTCA")

-41.27374490729283

In [26]:
# Testing on sequence
obs_seq = "CTTCATGTGAAAGCAGACGTAAGTCA"
most_likely_path = viterbi(obs_seq)
print("Most likely state path:")
print(most_likely_path)

Most likely state path:
EEEEEEEEEEEEEEEEEEEEEEEEEE
